In [139]:
# manipulando tabela do brasileirão série A a partir de 2012
# para verificação de estratégia de Poisson - ainda falta terminar

import csv 
import statistics as st
import math
import numpy as np

# Distribuição de Poisson para resultados de futebol
def poisson(time1,time2,mgf_ccasa,mgs_ccasa,mgf_cfora,mgs_cfora,rodada):
    mgf_tcasa = time1['mgf_tcasa']
    mgs_tcasa = time1['mgs_tcasa']
    mgf_tfora = time2['mgf_tfora']
    mgs_tfora = time2['mgf_tfora']
    
    # tratamento de zeros
    if mgf_tcasa == 0:
        mgf_tcasa = 0.1
        
    if mgs_tcasa == 0:
        mgs_tcasa = 0.1
        
    if mgf_tfora == 0:
        mgf_tfora = 0.1
        
    if mgs_tfora == 0:
        mgs_tfora = 0.1
        
    if mgf_ccasa == 0:
        mgf_ccasa = 0.1
        
    if mgs_ccasa == 0:
        mgs_ccasa = 0.1
        
    if mgf_cfora == 0:
        mgf_cfora = 0.1
        
    if mgs_cfora == 0:
        mgs_cfora = 0.1
    
    media_getc = mgf_tcasa / mgf_ccasa * mgs_tfora / mgs_cfora * mgf_ccasa
    media_getf = mgf_tfora / mgf_cfora * mgs_tcasa / mgs_ccasa * mgf_cfora

    # p/ prob. do número de gols
    probf_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probf_fora = [0,0,0,0,0,0,0,0,0,0,0]
    probs_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probs_fora = [0,0,0,0,0,0,0,0,0,0,0]

    # p/ prob. de vitória, empate e derrota
    probv_casa = 0
    probv_fora = 0
    prob_empate = 0

    # p/ prob. dupla chance
    dupla_casa = 0
    dupla_fora = 0

    # p/ prob. empate anula
    emp_ac = 0
    emp_af = 0

    # p/ prob. under e over
    under_05 = 0
    over_05 = 0
    under_15 = 0
    over_15 = 0
    under_25 = 0
    over_25 = 0
    under_35 = 0
    over_35 = 0

    # p/ prob. ambas marcam
    ambas_sim = 0
    ambas_nao = 0

    prob_placar = np.zeros((11,11), dtype=np.float64)
    placar_ordenado = []
    pos_placar_ordenado = []
    posicao = 0
    cont = 0

    # prob. de gols do time da casa e fora
    for i in range(11):
        probf_casa[i] = (math.exp(-media_getc)*media_getc**i)/math.factorial(i)
        probf_fora[i] = (math.exp(-media_getf)*media_getf**i)/math.factorial(i)

    # prob. de placares do jogo; prob. de 1x2; prob. de under e over
    # i -> num de gols do time da casa; j -> num de gols do time de fora
    for i in range(11):
        for j in range(11):
            # prob. de 1x2
            if i > j:
                probv_casa += probf_casa[i] * probf_fora[j] * 100
            elif j > i:
                probv_fora += probf_casa[i] * probf_fora[j] * 100
            else: 
                prob_empate += probf_casa[i] * probf_fora[j] * 100

            # prob. de under e over
            # under/over 0.5
            if i + j <= 0:
                under_05 += probf_casa[i] * probf_fora[j] * 100

            # under/over 1.5
            if i + j <= 1:
                under_15 += probf_casa[i] * probf_fora[j] * 100

            # under/over 2.5
            if i + j <= 2:
                under_25 += probf_casa[i] * probf_fora[j] * 100

            # under/over 3.5
            if i + j <= 3:
                under_35 += probf_casa[i] * probf_fora[j] * 100

            # prob. ambas marcam
            if (i > 0) and (j > 0):
                ambas_sim += probf_casa[i] * probf_fora[j] * 100

            # prob. de placar exato
            prob_placar[i][j] = probf_casa[i] * probf_fora[j] * 100
            placar_ordenado.append(prob_placar[i][j])
            pos_placar_ordenado.append(placar_ordenado[cont])
            cont += 1
            # print(f'{i} x {j} -> {prob_placar[i][j]:.2f}%')

    # prob. dupla chance
    dupla_casa = (probv_casa + prob_empate) 
    dupla_fora = (probv_fora + prob_empate)

    # prob. empate anula
    emp_ac = probv_casa/(probv_casa + probv_fora) * 100
    emp_af = 100 - emp_ac

    # prob. over 0.5
    over_05 = 100 - under_05

    # prob. over 1.5
    over_15 = 100 - under_15

    # prob. over 2.5
    over_25 = 100 - under_25

    # prob. over 3.5
    over_35 = 100 - under_35

    # prob. ambas marcam não
    ambas_nao = 100 - ambas_sim

    placar_ordenado.sort(reverse = True)
    
    print(f'Rodada {rodada}')
    print('{} x {}'.format(time1['time'], time2['time']))
    print(f'prob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%\n')


    '''print(f'\nPlacar Ordenado\n')
    # for i in range(121): 121 é o máximo. Nesse range pode-se escolher quantos resultados corretos exibir
    for i in range(10):
        posicao = pos_placar_ordenado.index(placar_ordenado[i])

        # Resultados Corretos
        print(f'{int(posicao/11)} x {posicao%11} -> {placar_ordenado[i]:.2f}%')

    # 1x2
    print(f'\nprob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%')

    # dupla chance
    print(f'\nprob. dupla chance casa => {dupla_casa:.2f}%')
    print(f'prob. dupla chance visitante => {dupla_fora:.2f}%')

    # empate anula
    print(f'\nprob. empate anula casa => {emp_ac:.2f}%')
    print(f'prob. empate anula visitante: => {emp_af:.2f}%')

    # under/over 0.5
    print(f'\nprob. under 0.5 => {under_05:.2f}%')
    print(f'prob. over 0.5 => {over_05:.2f}%')

    # under/over 1.5
    print(f'\nprob. under 1.5 => {under_15:.2f}%')
    print(f'prob. over 1.5 => {over_15:.2f}%')

    # under/over 2.5
    print(f'\nprob. under 2.5 => {under_25:.2f}%')
    print(f'prob. over 2.5 => {over_25:.2f}%')

    # under/over 3.5
    print(f'\nprob. under 3.5 => {under_35:.2f}%')
    print(f'prob. over 3.5 => {over_35:.2f}%')

    # ambas marcam
    print(f'\nprob. ambas marcam sim => {ambas_sim:.2f}%')
    print(f'prob. ambas marcam não => {ambas_nao:.2f}%')'''
    pass

    
# base de dados
time_casa = ''
time_fora = ''
num_jogos = 0
num_gols = 0
media_gols = 0

mgf_tcasa = 0
mgs_tcasa = 0
mgf_tfora = 0
mgs_tfora = 0
mgf_ccasa = 0
mgs_ccasa = 0
mgf_cfora = 0
mgs_cfora = 0
time = [''] * 20

tgf_ccasa = 0
tgf_cfora = 0

cont1 = 0
cont_jogos = 0
rodada = 1
cont_geral = 1
cont_ind = [0] * 20

time1 = ''
time2 = ''
dentro_rodada = 0


with open('BRA2.csv', 'r', encoding = 'utf-8', newline = '') as al:
    arq1 = csv.DictReader(al)
    
    for linha in arq1:  
        # ano de 2012
        if (linha['Season'] == '2018'): 
        
            # media de gols dos times e do campeonato
            tgf_ccasa += float(linha['HG'])
            tgf_cfora += float(linha['AG'])
            mgf_ccasa = tgf_ccasa/cont_geral
            mgs_ccasa = tgf_cfora/cont_geral
            mgf_cfora = mgs_ccasa
            mgs_cfora = mgf_ccasa
            
            # carregando os times e o número de jogos de cada
            if cont1 < 10:
                cont_ind[cont1] += 1
                cont_ind[cont1+10] += 1
                time[cont1] = {'time':linha['Home'],'num_jogosc':cont_ind[cont1],'num_jogosf':0,'tgf_tcasa':int(linha['HG']),'tgs_tcasa':int(linha['AG']),'tgf_tfora':0,'tgs_tfora':0,'mgf_tcasa':float(linha['HG'])/cont_ind[cont1],'mgs_tcasa':float(linha['AG'])/cont_ind[cont1],'mgf_tfora':0,'mgs_tfora':0}
                time[cont1+10] = {'time':linha['Away'],'num_jogosc':0,'num_jogosf':cont_ind[cont1+10],'tgf_tcasa':0,'tgs_tcasa':0,'tgf_tfora':int(linha['AG']),'tgs_tfora':int(linha['HG']),'mgf_tcasa':0,'mgs_tcasa':0,'mgf_tfora':float(linha['AG'])/cont_ind[cont1+10],'mgs_tfora':float(linha['HG'])/cont_ind[cont1+10]}
                cont1 += 1
                
            # contando jogos
            if cont_jogos <= 10:
                cont_jogos += 1
                
            # atualizando os dados dos times por jogo
            if cont_geral >= 10:     
                for i in range(20):
                    if linha['Home'] == time[i]['time']: 
                        time[i]['num_jogosc'] += 1
                        time[i]['tgf_tcasa'] += int(linha['HG'])
                        time[i]['tgs_tcasa'] += int(linha['AG'])
                       
                        time[i]['mgf_tcasa'] = time[i]['tgf_tcasa']/time[i]['num_jogosc']
                        time[i]['mgs_tcasa'] = time[i]['tgs_tcasa']/time[i]['num_jogosc']
                        
                    if linha['Away'] == time[i]['time']:
                        time[i]['num_jogosf'] += 1
                        time[i]['tgf_tfora'] += int(linha['AG'])
                        time[i]['tgs_tfora'] += int(linha['HG'])

                        time[i]['mgf_tfora'] = time[i]['tgf_tfora']/time[i]['num_jogosf']
                        time[i]['mgs_tfora'] = time[i]['tgs_tfora']/time[i]['num_jogosf']
            
                                         
            #contando rodadas - a primeira vez que entra aqui é a rodada 2
            if cont_jogos > 10 or dentro_rodada < 10:
                # atualizando o número de rodadas
                if cont_jogos > 10:
                    rodada += 1
                    cont_jogos = 1
                    dentro_rodada = 0
                    
                dentro_rodada += 1
                
                #calculando dados da rodada
                if rodada > 1:
                    for i in range(20):                 
                        if linha['Home'] == time[i]['time']:
                            time1 = time[i]
                        if linha['Away'] == time[i]['time']:
                            time2 = time[i]
                            
                if rodada > 1:
                    poisson(time1,time2,mgf_ccasa,mgs_ccasa,mgf_cfora,mgs_cfora,rodada)

            cont_geral += 1

Rodada 2
Bahia x Santos
prob. de vitória casa => 5.03%
prob. de vitória visitante => 1.48%
prob. de empate: 93.49%

Rodada 2
Flamengo RJ x America MG
prob. de vitória casa => 9.75%
prob. de vitória visitante => 1.53%
prob. de empate: 88.72%

Rodada 2
Parana x Corinthians
prob. de vitória casa => 0.00%
prob. de vitória visitante => 1.92%
prob. de empate: 0.00%

Rodada 2
Atletico-MG x Vitoria
prob. de vitória casa => 34.62%
prob. de vitória visitante => 36.93%
prob. de empate: 28.46%

Rodada 2
Ceara x Sao Paulo
prob. de vitória casa => 0.59%
prob. de vitória visitante => 1.23%
prob. de empate: 98.17%

Rodada 2
Chapecoense-SC x Vasco
prob. de vitória casa => 18.57%
prob. de vitória visitante => 51.23%
prob. de empate: 30.20%

Rodada 2
Fluminense x Cruzeiro
prob. de vitória casa => 6.03%
prob. de vitória visitante => 1.22%
prob. de empate: 92.75%

Rodada 2
Palmeiras x Internacional
prob. de vitória casa => 6.14%
prob. de vitória visitante => 1.29%
prob. de empate: 92.57%

Rodada 2
Gremio x

Rodada 18
Santos x Sport Recife
prob. de vitória casa => 34.81%
prob. de vitória visitante => 33.57%
prob. de empate: 31.62%

Rodada 18
Corinthians x Gremio
prob. de vitória casa => 31.29%
prob. de vitória visitante => 19.78%
prob. de empate: 48.93%

Rodada 19
Atletico-PR x Flamengo RJ
prob. de vitória casa => 44.65%
prob. de vitória visitante => 34.33%
prob. de empate: 21.02%

Rodada 19
Internacional x Parana
prob. de vitória casa => 19.45%
prob. de vitória visitante => 4.15%
prob. de empate: 76.40%

Rodada 19
Botafogo RJ x Atletico-MG
prob. de vitória casa => 28.78%
prob. de vitória visitante => 47.68%
prob. de empate: 23.54%

Rodada 19
Cruzeiro x Bahia
prob. de vitória casa => 20.21%
prob. de vitória visitante => 22.33%
prob. de empate: 57.46%

Rodada 19
Vitoria x Palmeiras
prob. de vitória casa => 20.34%
prob. de vitória visitante => 57.16%
prob. de empate: 22.49%

Rodada 19
America MG x Fluminense
prob. de vitória casa => 30.55%
prob. de vitória visitante => 38.32%
prob. de empate

Rodada 36
Flamengo RJ x Gremio
prob. de vitória casa => 44.83%
prob. de vitória visitante => 12.25%
prob. de empate: 42.91%

Rodada 36
Palmeiras x America MG
prob. de vitória casa => 39.38%
prob. de vitória visitante => 15.98%
prob. de empate: 44.64%

Rodada 36
Vasco x Sao Paulo
prob. de vitória casa => 27.89%
prob. de vitória visitante => 49.45%
prob. de empate: 22.66%

Rodada 36
Bahia x Fluminense
prob. de vitória casa => 36.72%
prob. de vitória visitante => 28.66%
prob. de empate: 34.62%

Rodada 36
Ceara x Parana
prob. de vitória casa => 11.25%
prob. de vitória visitante => 18.26%
prob. de empate: 70.49%

Rodada 36
Chapecoense-SC x Sport Recife
prob. de vitória casa => 28.83%
prob. de vitória visitante => 38.46%
prob. de empate: 32.71%

Rodada 37
Santos x Atletico-MG
prob. de vitória casa => 40.90%
prob. de vitória visitante => 32.75%
prob. de empate: 26.35%

Rodada 37
Atletico-PR x Ceara
prob. de vitória casa => 49.13%
prob. de vitória visitante => 20.56%
prob. de empate: 30.31%

R

In [ ]:
def t(c1):
    print(c1['novo'])


cont = [0] * 3
c1 = {'conta':cont[0],'novo':cont[0]}
c1['conta'] = 3
c1['novo'] = c1['conta'] + 3
t(c1)